In [1]:
import torch
import torch.nn as nn
import os
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
import torchvision.models as models

from PIL import Image
from sympy.polys.agca.modules import Module
from torch.distributions import transform_to

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2

train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [2]:
#猫狗分类数据集
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.Data = []
        self.Label = []
        self.transform = transform
        self.data_dir = data_dir
        # 读取数据
        for file in os.listdir(data_dir):
            self.Data.append(file)
            if 'cat' in file:
                self.Label.append(0)
            else:
                self.Label.append(1)

    def __len__(self):
        return len(self.Data)

    def __getitem__(self, idx):
        img = Image.open(self.data_dir + '/' + self.Data[idx])
        label = self.Label[idx]
        if self.transform:
            img = self.transform(img)
        return img, label




In [3]:
train_dir = 'data/train'

train_data = MyDataset(train_dir, transform=train_transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True)

In [4]:
model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)

model.fc = nn.Linear(512, num_classes)

In [5]:
model.load_state_dict(torch.load('cat_dog.pth'))
model.to(device)

In [ ]:
# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.0001)

# num_epochs = 10

# for epoch in range(num_epochs):
#     model.train()
#     for i,(x,y) in enumerate(train_loader):
#         x = x.to(device)
#         y = y.to(device)
#         optimizer.zero_grad()
#         outputs = model(x)
#         loss = criterion(outputs, y)
#         loss.backward()
#         optimizer.step()
#         print('epoch{} loss:{}'.format(epoch, loss.item()))

# torch.save(model.state_dict(), 'cat_dog.pth')

In [7]:
val_dir = 'data/val'

val_data = MyDataset(val_dir, transform=train_transform)

val_loader = torch.utils.data.DataLoader(val_data, batch_size=128, shuffle=False)

In [8]:

total = 0
correct = 0
model.eval()
for i,(x,y) in enumerate(val_loader):
    x = x.to(device)
    y = y.to(device)
    outputs = model(x)
    _, predicted = torch.max(outputs.data, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()
print(correct / total)

0.969


In [9]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None):
        self.Data = []
        self.Label = []
        self.data_dir = data_dir
        self.transform = transform
        for i in range(2000):
            self.Data.append(str(i)+'.jpg')
            self.Label.append(str(i))
    def __len__(self):
        return len(self.Data)

    def __getitem__(self, idx):
        img = Image.open(test_dir + '/' + self.Data[idx])
        if self.transform:
            img = self.transform(img)
        label = self.Label[idx]
        return img,label

In [10]:
test_dir = 'data/test'

testDataset = TestDataset(test_dir,train_transform)

test_loader = torch.utils.data.DataLoader(testDataset, batch_size=1, shuffle=False)

In [11]:
model.to(device)
model.eval()
result = []
label = []
for i,(x,y) in enumerate(test_loader):
    x = x.to(device)
    label.append(y[0])
    y = model(x).cpu()
    _,predicted = torch.max(y,1)
    result.append(predicted.tolist()[0])

In [12]:
import csv
def toCSV(a, b, filename):
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        for i in range(len(a)):
            writer.writerow([a[i], b[i]])

In [14]:
toCSV(label,result,'cat_dog.csv')